In [25]:
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\peiyi\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

C:\Users\peiyi\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
# examine the first 5 rows of X (including the feature names)
import pandas as pd
# Splitting data into training/testing
from sklearn.model_selection import train_test_split
df =  pd.read_csv("df_data_Global_final.csv")
df.head()

C:\Users\peiyi\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,Unnamed: 0.1,created_at,text,source,name,username,location,description,location2,coordinates,state,country,text_clean,sentiment,sentiment_1
0,0,8,2022-04-07 23:59:23+00:00,"Sigh and wonder how the decimal on my ""concurr...",TweetDeck,Fusionx,rofldrg,"Seattle, WA",andrew copeland content podcast producer https...,"Seattle, King County, Washington, United States","(47.6038321, -122.3300624, 0.0)",Seattle,United States,sigh and wonder how the decimal on my concurre...,neg,0
1,1,16,2022-04-07 23:58:50+00:00,"🍔: Well, guess I should let y'all know I've be...",Twitter for Android,Aaron Schneider,aschneider_96,"Kansas City, MO",civil engineer university nebraska alumni,"Kansas City, Jackson County, Missouri, United ...","(39.100105, -94.5781416, 0.0)",Kansas City,United States,: well guess i should let y'all know i've been...,neg,0
2,2,19,2022-04-07 23:58:11+00:00,@SEGAbits McDonald's now with the depression meal,Twitter Web App,Dr. Ewa J. Antczak,Core_IdealMe,"Mississauga, Ontario",psychologist founder idealme enrichment founda...,"Mississauga, Peel Region, Golden Horseshoe, On...","(43.5896231, -79.6443879, 0.0)",Mississauga,Canada,segabits mcdonald's now with the depression meal,neg,0
3,3,21,2022-04-07 23:58:03+00:00,@Amer1can_Barbie There is a huge difference be...,Twitter for iPhone,"Andrew, Cymro, 🏴󠁧󠁢󠁷󠁬󠁳󠁿🇪🇺🇺🇦🇵🇸, #NHS💙",_Annibyniaeth,"Caerphilly, Wales",dm invite covid x husband father retired life ...,"Cymru / Wales, United Kingdom","(52.2928116, -3.73893, 0.0)",Cymru / Wales,United Kingdom,amercanbarbie there is a huge difference betwe...,neg,0
4,4,34,2022-04-07 23:56:59+00:00,depression tucks me in at 7pm sharp every night,Twitter for iPhone,Bubbles,BigBankBubbles_,"Atlanta, GA",go beach bring back seashell longliveni,"Atlanta, Fulton County, Georgia, United States","(33.7489924, -84.3902644, 0.0)",Atlanta,United States,depression tucks me in at pm sharp every night,neg,0


In [28]:
# Convert to list 
data = df['text_clean'].values.tolist()  
# Remove Emails 
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]  
# Remove new line characters 
data = [re.sub('\s+', ' ', sent) for sent in data]  
# Remove distracting single quotes 
data = [re.sub("\'", "", sent) for sent in data]  
pprint(data[:1])

C:\Users\peiyi\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:4: DeprecationWarning: invalid escape sequence \S
<>:6: DeprecationWarning: invalid escape sequence \s
<>:4: DeprecationWarning: invalid escape sequence \S
<>:6: DeprecationWarning: invalid escape sequence \s
<ipython-input-28-4d3f5009bb6a>:4: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
<ipython-input-28-4d3f5009bb6a>:6: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


['sigh and wonder how the decimal on my concurrent viewers stat will change in '
 'a way that continues to impede me from affiliate status which then usually '
 'results in depression and losing interest in streaming for a few weeks ']


In [29]:
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

C:\Users\peiyi\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['sigh', 'and', 'wonder', 'how', 'the', 'decimal', 'on', 'my', 'concurrent', 'viewers', 'stat', 'will', 'change', 'in', 'way', 'that', 'continues', 'to', 'impede', 'me', 'from', 'affiliate', 'status', 'which', 'then', 'usually', 'results', 'in', 'depression', 'and', 'losing', 'interest', 'in', 'streaming', 'for', 'few', 'weeks']]


In [30]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

C:\Users\peiyi\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['sigh', 'and', 'wonder', 'how', 'the', 'decimal', 'on', 'my', 'concurrent', 'viewers', 'stat', 'will', 'change', 'in', 'way', 'that', 'continues', 'to', 'impede', 'me', 'from', 'affiliate', 'status', 'which', 'then', 'usually', 'results', 'in', 'depression', 'and', 'losing', 'interest', 'in', 'streaming', 'for', 'few_weeks']


In [31]:
# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

C:\Users\peiyi\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

C:\Users\peiyi\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['wonder', 'decimal', 'concurrent', 'viewer', 'stat', 'change', 'way', 'continue', 'affiliate', 'status', 'usually', 'result', 'depression', 'lose', 'interest', 'streaming', 'week']]


In [36]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

C:\Users\peiyi\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)]]


In [37]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

C:\Users\peiyi\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[('affiliate', 1),
  ('change', 1),
  ('concurrent', 1),
  ('continue', 1),
  ('decimal', 1),
  ('depression', 1),
  ('interest', 1),
  ('lose', 1),
  ('result', 1),
  ('stat', 1),
  ('status', 1),
  ('streaming', 1),
  ('usually', 1),
  ('viewer', 1),
  ('way', 1),
  ('week', 1),
  ('wonder', 1)]]

In [47]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

C:\Users\peiyi\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.027*"say" + 0.019*"good" + 0.019*"would" + 0.017*"amp" + 0.015*"take" + '
  '0.014*"s" + 0.012*"work" + 0.012*"lose" + 0.011*"never" + 0.011*"love"'),
 (1,
  '0.169*"sad" + 0.042*"be" + 0.017*"make" + 0.017*"go" + 0.016*"see" + '
  '0.015*"really" + 0.014*"get" + 0.012*"know" + 0.012*"feel" + 0.010*"life"'),
 (2,
  '0.027*"people" + 0.022*"stress" + 0.022*"time" + 0.020*"think" + '
  '0.016*"day" + 0.013*"man" + 0.013*"well" + 0.012*"still" + 0.010*"try" + '
  '0.008*"hear"')]


C:\Users\peiyi\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [53]:
# Visualize the topics
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\peiyi\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.307532 -0.009003       1        1  47.792778
2      0.162758 -0.226427       2        1  29.343572
0      0.144774  0.235430       3        1  22.863650, topic_info=        Term          Freq         Total Category  logprob  loglift
411      sad  20535.000000  20535.000000  Default  30.0000  30.0000
59        be   5072.000000   5072.000000  Default  29.0000  29.0000
126   people   2049.000000   2049.000000  Default  28.0000  28.0000
203      say   1555.000000   1555.000000  Default  27.0000  27.0000
358   stress   1655.000000   1655.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
678   family    420.058730    476.413359   Topic3  -4.9313   1.3497
1110   leave    524.942995    665.871812   Topic3  -4.7084   1.2378
571     love    621.915525    962.030979   Topic3  -4.5389   1.0394
173        s    815.734606   1877.597412   Topic3  -4.2676   0.6420
2352    news    308.128378    339.580279   Topic3  -5.2412   1.3784

[146 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
159       2  0.001679  actually
159       3  0.998744  actually
580       1  0.999061      also
665       1  0.957435    always
665       2  0.034839    always
...     ...       ...       ...
133       2  0.002520      work
133       3  0.861961      work
330       2  0.998108     world
118       3  0.998739     would
53        1  0.998731      year

[181 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])